<a href="https://colab.research.google.com/github/danielsaniam/ai-recipe-finder/blob/main/IamCooking_Open_AI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -U sentence-transformers
!pip install pinecone-client
!pip install gradio
!pip install datasets
!pip install openai
!pip install tiktoken

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 1,000.0 kB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 8.6 MB/s eta 0:00:00
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125923 sha256=a03181a134a0771b22ad9ff1dd8bce535f57421d5ea36874de66ae109ca1cc61
  Stored in directory: /root/.cache/pip/wheels/62/f2/10/1e606fd5f02395388f74e7462910fe851042f97238cbbd902f
Successfully built sentence-transformers
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.4/179.4 kB 3.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 7.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 300.4/300.4 kB 10.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.6/16.6 MB 43.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.0/92.0 kB 14.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━

In [ ]:
import pandas as pd
from sentence_transformers import SentenceTransformer, util
from ast import literal_eval
from datasets import load_dataset
from openai import OpenAI
from getpass import getpass
from openai import OpenAI

In [ ]:
dataset = load_dataset("somosnlp/recetas-cocina")

Generating train split: 0 examples [00:00, ? examples/s]

In [ ]:
df = pd.DataFrame(dataset['train'])

In [ ]:
df.head(10)

,title,url,ingredients,steps,uuid
0,Arepas de Queso,https://www.mycolombianrecipes.com/es/arepas-d...,1 taza de harina de arepa blanca o amarilla\r\...,"Combine la harina de maíz, agua caliente, el q...",86af61e4-e16a-11ed-9591-a96d6180cd25
1,Sudado de Pollo,https://www.mycolombianrecipes.com/es/sudado-d...,8 muslos de pollo sin la piel\r\n1 cucharada d...,"En una olla grande, caliente el aceite vegetal...",86af61e5-e16a-11ed-abef-a96d6180cd25
2,Sancocho Trifásico,https://www.mycolombianrecipes.com/es/sancocho...,1 taza de cebolla picada\r\n1 pimientón rojo f...,"Coloque la cebolla, el pimientón, el ajo y el ...",86af61e6-e16a-11ed-bcf5-a96d6180cd25
3,Sudado de Posta o Muchacho,https://www.mycolombianrecipes.com/es/sudado-d...,"1 posta o muchacho de 3 a 4 libras, cortada en...","En una olla grande ponga la carne, el caldo, ...",86af61e7-e16a-11ed-8914-a96d6180cd25
4,Sopa de Lentejas Colombiana,https://www.mycolombianrecipes.com/es/sopa-de-...,1 cucharadita de aceite vegetal\r\n1 taza de c...,Cocinar el chorizo en el aceite en una olla gr...,86af71bc-e16a-11ed-b130-a96d6180cd25
5,Camarones con Salsa de Cilantro,https://www.mycolombianrecipes.com/es/camarone...,4 cucharadas de aceite de oliva\r\n4 dientes d...,"En una sartén a fuego medio, caliente el aceit...",86af71bd-e16a-11ed-be2a-a96d6180cd25
6,Bistec a la Criolla,https://www.mycolombianrecipes.com/es/bistec-a...,"2 libras de carne de bistec falda, cortado en ...","Coloque los filetes entre plastico, golpee has...",86af71be-e16a-11ed-afa4-a96d6180cd25
7,Arepas de Yuca,https://www.mycolombianrecipes.com/es/arepas-d...,1 libra de yuca cocida\r\nMedia taza de Masare...,"Triturar la yuca y agregue la masarepa, la lec...",86af71bf-e16a-11ed-b6df-a96d6180cd25
8,Sopa de Lentejas con Carne,https://www.mycolombianrecipes.com/es/sopa-de-...,1 taza de salsa de aliños ver receta aquí\r\n2...,"Ponga la salsa de aliños, zanahorias, carne de...",86af71c0-e16a-11ed-924d-a96d6180cd25
9,Bistec Encebollado,https://www.mycolombianrecipes.com/es/bistec-e...,2 libras de carne de falda o filete\r\n3 dient...,"Coloque los filetes entre plastico, machaque h...",86af71c1-e16a-11ed-9673-a96d6180cd25


In [ ]:
def concatenar_lista(lista):
    lista = literal_eval(lista)
    return ' '.join(lista)

In [ ]:
def string_to_list(lista):
    lista = literal_eval(lista)
    return lista

In [ ]:
df = df.fillna(' ')
df['text'] = df.apply(lambda x : x['title']+' '+x['ingredients']+' '+x['steps'], axis=1)

In [ ]:
df.head(3)

,title,url,ingredients,steps,uuid,text
0,Arepas de Queso,https://www.mycolombianrecipes.com/es/arepas-d...,1 taza de harina de arepa blanca o amarilla\r\...,"Combine la harina de maíz, agua caliente, el q...",86af61e4-e16a-11ed-9591-a96d6180cd25,Arepas de Queso 1 taza de harina de arepa blan...
1,Sudado de Pollo,https://www.mycolombianrecipes.com/es/sudado-d...,8 muslos de pollo sin la piel\r\n1 cucharada d...,"En una olla grande, caliente el aceite vegetal...",86af61e5-e16a-11ed-abef-a96d6180cd25,Sudado de Pollo 8 muslos de pollo sin la piel\...
2,Sancocho Trifásico,https://www.mycolombianrecipes.com/es/sancocho...,1 taza de cebolla picada\r\n1 pimientón rojo f...,"Coloque la cebolla, el pimientón, el ajo y el ...",86af61e6-e16a-11ed-bcf5-a96d6180cd25,Sancocho Trifásico 1 taza de cebolla picada\r\...


## Evaluar cantidad de tokens a procesar

---



In [ ]:
import tiktoken

def num_tokens_from_string(text, encodig_name):
    encoding = tiktoken.get_encoding(encodig_name)
    num_tokens = len(encoding.encode(text))
    return num_tokens


In [ ]:
df['total_tokens'] = df['text'].apply(lambda x : num_tokens_from_string(x,'cl100k_base') )

In [ ]:
sum(df['total_tokens'])

11394175

In [ ]:
OPENAI_API_KEY = getpass('Enter the secret value: ')
client_openai = OpenAI(
  api_key=OPENAI_API_KEY
)

In [ ]:
def get_embedding(text, model="text-embedding-ada-002"):
   text = text.replace("\n", " ")
   return client_openai.embeddings.create(input = [text], model=model).data[0].embedding

In [ ]:
df['embeddings'] = ''

In [ ]:
df[df['embeddings'] != '']

,title,url,ingredients,steps,uuid,text,total_tokens,embeddings


In [ ]:
empty_embeddings = df[df['embeddings'] == '']

In [ ]:
empty_embeddings

,title,url,ingredients,steps,uuid,text,total_tokens,embeddings
0,Arepas de Queso,https://www.mycolombianrecipes.com/es/arepas-d...,1 taza de harina de arepa blanca o amarilla\r\...,"Combine la harina de maíz, agua caliente, el q...",86af61e4-e16a-11ed-9591-a96d6180cd25,Arepas de Queso 1 taza de harina de arepa blan...,215,
1,Sudado de Pollo,https://www.mycolombianrecipes.com/es/sudado-d...,8 muslos de pollo sin la piel\r\n1 cucharada d...,"En una olla grande, caliente el aceite vegetal...",86af61e5-e16a-11ed-abef-a96d6180cd25,Sudado de Pollo 8 muslos de pollo sin la piel\...,326,
2,Sancocho Trifásico,https://www.mycolombianrecipes.com/es/sancocho...,1 taza de cebolla picada\r\n1 pimientón rojo f...,"Coloque la cebolla, el pimientón, el ajo y el ...",86af61e6-e16a-11ed-bcf5-a96d6180cd25,Sancocho Trifásico 1 taza de cebolla picada\r\...,397,
3,Sudado de Posta o Muchacho,https://www.mycolombianrecipes.com/es/sudado-d...,"1 posta o muchacho de 3 a 4 libras, cortada en...","En una olla grande ponga la carne, el caldo, ...",86af61e7-e16a-11ed-8914-a96d6180cd25,Sudado de Posta o Muchacho 1 posta o muchacho ...,288,
4,Sopa de Lentejas Colombiana,https://www.mycolombianrecipes.com/es/sopa-de-...,1 cucharadita de aceite vegetal\r\n1 taza de c...,Cocinar el chorizo en el aceite en una olla gr...,86af71bc-e16a-11ed-b130-a96d6180cd25,Sopa de Lentejas Colombiana 1 cucharadita de a...,372,
...,...,...,...,...,...,...,...,...
28233,Berenjenas rellenas de carne picada y bechamel...,https://www.recetasgratis.net/receta-de-berenj...,4 Berenjenas 2 Zanahorias 1 Cebolla 1 diente d...,1 Empezaremos a preparar estas berenjenas rell...,b0a6454b-a023-44af-a189-92dee8948490,Berenjenas rellenas de carne picada y bechamel...,781,
28234,Puré de patatas con queso y huevo,https://www.recetasgratis.net/receta-de-pure-d...,2 Patatas 2 Huevos 100 gramos de Parmesano 2 l...,"1 Cocemos las patatas y, una vez listas, las p...",6e52da68-9496-47de-b85c-7be7fd0b6ac2,Puré de patatas con queso y huevo 2 Patatas 2 ...,411,
28235,Rollitos de calabacín rellenos de jamón y queso,https://www.recetasgratis.net/receta-de-rollit...,½ Calabacín 100 gramos de Jamón cocido 100 gra...,1 Lo primero que debemos hacer es lavar bien e...,22412360-37fe-47de-9401-d3f1e95fe4e7,Rollitos de calabacín rellenos de jamón y ques...,417,
28236,Calabacín relleno de quinoa,https://www.recetasgratis.net/receta-de-calaba...,4 Calabacines 1 Cebolla 1 Pimiento verde 1 Pim...,1 Lo primero que tenemos que hacer para elabor...,eedfabb6-d14f-46e4-a34f-dc07b9015636,Calabacín relleno de quinoa 4 Calabacines 1 Ce...,726,


In [ ]:
empty_embeddings['embeddings'] = empty_embeddings.apply(lambda row: get_embedding(row['text'], model='text-embedding-ada-002'), axis=1)

RateLimitError: ignored

In [ ]:
df.head(3)

,title,url,ingredients,steps,uuid,text,total_tokens,embeddings
0,Arepas de Queso,https://www.mycolombianrecipes.com/es/arepas-d...,1 taza de harina de arepa blanca o amarilla\r\...,"Combine la harina de maíz, agua caliente, el q...",86af61e4-e16a-11ed-9591-a96d6180cd25,Arepas de Queso 1 taza de harina de arepa blan...,215,
1,Sudado de Pollo,https://www.mycolombianrecipes.com/es/sudado-d...,8 muslos de pollo sin la piel\r\n1 cucharada d...,"En una olla grande, caliente el aceite vegetal...",86af61e5-e16a-11ed-abef-a96d6180cd25,Sudado de Pollo 8 muslos de pollo sin la piel\...,326,
2,Sancocho Trifásico,https://www.mycolombianrecipes.com/es/sancocho...,1 taza de cebolla picada\r\n1 pimientón rojo f...,"Coloque la cebolla, el pimientón, el ajo y el ...",86af61e6-e16a-11ed-bcf5-a96d6180cd25,Sancocho Trifásico 1 taza de cebolla picada\r\...,397,


In [ ]:
import pinecone
from getpass import getpass

In [ ]:
pincone_api = getpass('Enter the secret value: ')

Enter the secret value: ··········


In [ ]:
pinecone.init(api_key=pincone_api, environment="gcp-starter")

In [ ]:
dimensions_embeddings = len(df['embeddings'][0])
index_name = 'recipes-embeddings-openai'
all_index = pinecone.list_indexes()
if index_name in all_index:
    index = pinecone.Index(index_name)
else:
    pinecone.create_index(index_name, dimension=dimensions_embeddings, metric="cosine")
    index = pinecone.Index(index_name)

In [ ]:
index = pinecone.Index('recipes-embeddings-openai')

In [ ]:
from tqdm.auto import tqdm

# we will use batches of 64
batch_size = 64

for i in tqdm(range(0, len(df), batch_size)):
    # find end of batch
    i_end = min(i+batch_size, len(df))
    # extract batch
    batch = df[i:i_end]
    # generate embeddings for batch
    ids = batch['ids']
    emb = batch['embeddings']
    metadata = batch.drop(['ids','embeddings','text'],axis=1).to_dict('records')

    # add all to upsert list
    to_upsert = list(zip(ids, emb,metadata))
    # upsert/insert these records to pinecone
    _ = index.upsert(vectors=to_upsert)

# check that we have all vectors in index
index.describe_index_stats()

  0%|          | 0/442 [00:00<?, ?it/s]

{'dimension': 384,
 'index_fullness': 0.2688,
 'namespaces': {'': {'vector_count': 26880}},
 'total_vector_count': 26880}

In [ ]:
query = 'receta con pollo'
query_vector = model.encode(query).tolist()

responses = index.query(
  vector=query_vector,
  top_k=3,
  include_metadata=True,
  filter ={}
)

NameError: ignored

In [ ]:
responses

{'matches': [{'id': '14809',
              'metadata': {'ingredients': 'Arroz previamente remojado 1 día '
                                          'Claro de maíz Leche Canela en polvo '
                                          'Sal Azucar blanca',
                           'steps': '1 Poner el claro de maíz con algunos '
                                    'maíces en una olla 2 Moler el arroz '
                                    'remojado 3 Hechar el azúcar , el arroz '
                                    'remojado de un día ya molido ,la '
                                    'canela,la leche y la pizca de salRevolver '
                                    'todo asta tener una mescla espesa, se '
                                    'sirve en un posillo y tomar calienta',
                           'title': 'Arroz con maíz y canela',
                           'url': 'https://www.recetasgratis.net/receta-de-arroz-con-maiz-y-canela-51684.html',
                           'uuid': '177

In [ ]:
def search(query, top_k):
    query_vector = model.encode(query).tolist()

    responses = index.query(
        vector=query_vector,
        top_k=top_k,
        include_metadata=True,
        filter={}
    )

    # Format the responses for better display
    response_data = []
    for response in responses['matches']:
        response_data.append({
            'Title': response['metadata']['title'],
            'Ingredients': response['metadata']['ingredients'],
            'Steps': response['metadata']['steps'],
            'URL': response['metadata']['url'],
            'Score': response['score'],
        })

    df = pd.DataFrame(response_data)
    return df



In [ ]:
import gradio as gr

iface = gr.Interface(
    fn=search,
    inputs=[
        gr.Textbox(lines=5, placeholder="Escribe aquí tu consulta...", label="Consulta")

    ],
    outputs=gr.Dataframe(type="pandas", label="Resultados"),
    title="Buscador de recetas",
    description="Introduce tu consulta para buscar recetas.",
)

# Launch the interface
iface.launch()


/usr/local/lib/python3.10/dist-packages/gradio/utils.py:840: UserWarning: Expected 2 arguments for function <function search at 0x7b12ee682cb0>, received 1.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/gradio/utils.py:844: UserWarning: Expected at least 2 arguments for function <function search at 0x7b12ee682cb0>, received 1.
  warnings.warn(


Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://6abab3f62bd8993639.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
